In [5]:
%pip install pandas 
%pip install scikit-learn 
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import re
import ast
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def parse_genres(genre_str):
    if pd.isna(genre_str):
        return []
    try:
        genre_list = ast.literal_eval(genre_str)
        if isinstance(genre_list, list):
            return genre_list
        return []
    except:
        return []
    
def clean_text(text):
    if pd.isna(text):
        return ""
    
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    
    words = text.split()
    words = [
        lemmatizer.lemmatize(w)
        for w in words
        if w not in stop_words and len(w) > 2
    ]
    
    return " ".join(words)

df = pd.read_csv("movies_info.csv")

# 合并可用文本字段
df["text"] = (
    df["original_title"].fillna("") + " " +
    df["overview"].fillna("")
)

df["clean_text"] = df["text"].apply(clean_text)
df["genres_list"] = df["genres"].apply(parse_genres)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df["genres_list"])


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.9
)

X = tfidf.fit_transform(df["clean_text"])

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [10]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

model = OneVsRestClassifier(
    LinearSVC(
        class_weight="balanced",
        C=1.0
    )
)

model.fit(X_train, y_train)


,"estimator estimator: estimator objectA regressor or a classifier that implements :term:`fit`.When a classifier is passed, :term:`decision_function` will be usedin priority and it will fallback to :term:`predict_proba` if it is notavailable.When a regressor is passed, :term:`predict` is used.",LinearSVC(cla...ht='balanced')
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation: the `n_classes`one-vs-rest problems are computed in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: 0.20 `n_jobs` default changed from 1 to None",None
,"verbose verbose: int, default=0The verbosity level, if non zero, progress messages are printed.Below 50, the output is sent to stderr. Otherwise, the output is sentto stdout. The frequency of the messages increases with the verbositylevel, reporting all iterations at 10. See :class:`joblib.Parallel` formore details... versionadded:: 1.1",0
,"penalty penalty: {'l1', 'l2'}, default='l2'Specifies the norm used in the penalization. The 'l2'penalty is the standard used in SVC. The 'l1' leads to ``coef_``vectors that are sparse.",'l2'
,"loss loss: {'hinge', 'squared_hinge'}, default='squared_hinge'Specifies the loss function. 'hinge' is the standard SVM loss(used e.g. by the SVC class) while 'squared_hinge' is thesquare of the hinge loss. The combination of ``penalty='l1'``and ``loss='hinge'`` is not supported.",'squared_hinge'
,"dual dual: ""auto"" or bool, default=""auto""Select the algorithm to either solve the dual or primaloptimization problem. Prefer dual=False when n_samples > n_features.`dual=""auto""` will choose the value of the parameter automatically,based on the values of `n_samples`, `n_features`, `loss`, `multi_class`and `penalty`. If `n_samples` < `n_features` and optimizer supportschosen `loss`, `multi_class` and `penalty`, then dual will be set to True,otherwise it will be set to False... versionchanged:: 1.3 The `""auto""` option is added in version 1.3 and will be the default in version 1.5.",'auto'
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive.For an intuitive visualization of the effects of scalingthe regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"multi_class multi_class: {'ovr', 'crammer_singer'}, default='ovr'Determines the multi-class strategy if `y` contains more thantwo classes.``""ovr""`` trains n_classes one-vs-rest classifiers, while``""crammer_singer""`` optimizes a joint objective over all classes.While `crammer_singer` is interesting from a theoretical perspectiveas it is consistent, it is seldom used in practice as it rarely leadsto better accuracy and is more expensive to compute.If ``""crammer_singer""`` is chosen, the options loss, penalty and dualwill be ignored.",'ovr'
,"fit_intercept fit_intercept: bool, default=TrueWhether or not to fit an intercept. If set to True, the feature vectoris extended to include an intercept term: `[x_1, ..., x_n, 1]`, where1 corresponds to the intercept. If set to False, no intercept will beused in calculations (i.e. data is expected to be already centered).",True
,"intercept_scaling intercept_scaling: float, default=1.0When `fit_intercept` is True, the instance vector x becomes ``[x_1,..., x_n, intercept_scaling]``, i.e. a ""synthetic"" feature with aconstant value equal to `intercept_scaling` is appended to the instancevector. The intercept becomes intercept_scaling * synthetic featureweight. Note that liblinear internally penalizes the intercept,treating it like any other term in the feature vector. To reduce theimpact of the regularization on the intercept, the `intercept_scaling`parameter can be set to a value greater than 1; the higher the value of`intercept_scaling`, the lower 

In [11]:
from sklearn.metrics import f1_score

for c in [0.1, 0.5, 1, 2, 5]:
    clf = OneVsRestClassifier(
        LinearSVC(class_weight="balanced", C=c)
    )
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    score = f1_score(y_test, preds, average="micro")
    print(f"C={c} | Micro-F1={score:.3f}")


C=0.1 | Micro-F1=0.626
C=0.5 | Micro-F1=0.611
C=1 | Micro-F1=0.596
C=2 | Micro-F1=0.581


c:\self_download\anaconda\envs\NLP_PROJECT\Lib\site-packages\sklearn\svm\_base.py:1258: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


C=5 | Micro-F1=0.562


In [12]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(
    classification_report(
        y_test,
        y_pred,
        target_names=mlb.classes_,
        zero_division=0
    )
)


                 precision    recall  f1-score   support

         Action       0.61      0.68      0.64       431
      Adventure       0.44      0.60      0.51       276
      Animation       0.57      0.57      0.57       172
         Comedy       0.62      0.67      0.65       710
          Crime       0.55      0.60      0.57       293
          Drama       0.68      0.67      0.67       950
         Family       0.55      0.58      0.57       209
        Fantasy       0.51      0.53      0.52       209
        History       0.43      0.31      0.36       122
         Horror       0.62      0.63      0.63       288
          Music       0.51      0.34      0.41        56
        Mystery       0.37      0.39      0.38       204
        Romance       0.56      0.58      0.57       375
Science Fiction       0.68      0.68      0.68       222
       TV Movie       0.67      0.21      0.32        19
       Thriller       0.59      0.63      0.61       548
            War       0.60    

Since the task is formulated as a multi-label classification problem, traditional accuracy is not suitable.
Therefore, micro-averaged F1-score is used as the primary evaluation metric.
The proposed model achieves a Micro-F1 score of 0.60, indicating strong overall predictive performance.

In [13]:
import joblib

joblib.dump(model, "movie_genre_multilabel_svm.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")
joblib.dump(mlb, "genre_binarizer.pkl")

print("✅ Model, vectorizer, and label binarizer saved.")


✅ Model, vectorizer, and label binarizer saved.
